# Alejandro Paz Olalla
# Enrique Queipo de Llano Burgos

# Práctica 1. Resolución de problemas con búsqueda 
## Inteligencia Artificial I    2022/2023

La práctica está organizada en 4 partes:
* Parte I: se muestra a través de ejemplos resueltos cómo se representan algunos problemas clásicos como el de las jarras, el problema de los misioneros o el problema del ocho puzzle. 
* Parte II: se muestra el uso de los algoritmos de búsqueda exhaustiva (ciega y heurística) vistos en clase. 
* Parte III: aprenderemos a medir las propiedades de los algoritmos.
* Parte IV: se os dan problemas algo más complejos para resolver al menos 1.

En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  
Cuando termines los ejercicios entrega **este archivo en el campus**.  
Debes incluir al comienzo una celda de markdown con el nombre completo de los miembros del grupo y número de grupo.

Los comentarios razonados de los ejercicios son la parte más importante de esta práctica. 

**Fecha de entrega de la práctica completa: 18 de octubre** 

## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en representar estados y acciones, la comprobación de objetivo y el coste de las acciones.
La siguiente clase Problem (de la librería AIMA) representa este esquema general de cualquier problema de espacio de estados. Un problema concreto será una subclase de Problem, y requerirá implementar
* estado_inicial, es_estado_final(_), acciones(_), aplica(_,_), coste_de_aplicar_accion y eventualmente __init__ 

El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. 


 __Lo más sencillo para empezar es copiar en tu directorio de trabajo los archivos _search.py y utils.py_ de AIMA__

In [4]:
from search import Problem
import math

In [2]:
## Copiamos aquí la definición de la clase Problem de la librería AIMA para facilitar la explicación.

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Respecto a la versiòn original de AIMA hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo __definir un problema como subclase
de problema__. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [6]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        #accs es una lista que inicializamos vacía, comprobaremos las precondiciones y añadiremos en esta lista las acciones aplicables al estado.
        accs=list() 
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs
        # se devuelve en accs todas las acciones aplicables

    def result(self,estado,accion):
    # aplica una acción a un estado (esta función se llamará desde el algoritmo de búsqueda)
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [7]:
# Estado inicial
p =Jarras()
p.initial

(0, 0)

In [8]:
# Acciones 
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [9]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [10]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [11]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [12]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
# Aunque no es necesario puedes hacer tu propia implementación del problema 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema. En este caso el estado inicial y objetivo se pasan como parámetros'''
        Problem.__init__(self, initial, goal)
        # En este caso representamos cada accion con un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido. La comprobación de las precondiciones se realiza en una función auxiliar is_valid
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3 
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:  #si la canoa está en el lado 0
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [13]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

In [14]:
misioneros.initial

(3, 3, 0)

In [15]:
# Acciones aplicables en el estado inicial
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

#Por ejemplo, para resolver el problema de los misioneros con 
#el método de busqueda en anchura la llamada sería:  
from search import *
breadth_first_tree_search(misioneros).solution()

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [11]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if (pos_hueco) not in (0,3,6):
            accs.append("Mover hueco izquierda")
        if (pos_hueco) not in (2,5,8):
            accs.append("Mover hueco derecha")
                
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0) #busco la pos del hueco
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco + 3]
            l[pos_hueco + 3] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco - 1]
            l[pos_hueco - 1] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco + 1]
            l[pos_hueco + 1] = 0 
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state.""" 
        return 1
    
    ##EJERCICIO 3
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0        

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [17]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [18]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [19]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [20]:
p8.result(p8.initial,"Mover hueco abajo")
#Falla porque la accion no es aplicable al estado inicial

IndexError: list index out of range

In [21]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [22]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")
# Está definida como return 1

1

### EJERCICIO 2.  Usando la representación anterior representa el problema del puzle de 16 piezas.

In [99]:
class Quince_Puzzle(Problem): 
    """Problema a del 15-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 15 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2,3):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (12,13,14,15):
            accs.append("Mover hueco abajo")
        if (pos_hueco) not in (0,4,8,12):
            accs.append("Mover hueco izquierda")
        if (pos_hueco) not in (3,7,11,15):
            accs.append("Mover hueco derecha")
                
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0) #busco la pos del hueco
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-4]
            l[pos_hueco-4] = 0
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco + 4]
            l[pos_hueco + 4] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco - 1]
            l[pos_hueco - 1] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco + 1]
            l[pos_hueco + 1] = 0 
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state.""" 
        return 1
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
                    
        for i in range(len(state)-1,0,-1):
                if (state[i] == 0):
                    fila = i // 4
                    
        if (fila % 2 == 0):
            return (inversion % 2 == 1)
        else:
            return (inversion % 2 == 0)

Realiza algunas pruebas para comprobar que la clase está bien definidA P

In [100]:
p15 = Quince_Puzzle((3, 14, 5 ,6, 9, 13, 0, 2, 8, 1, 12, 4, 11, 7, 10, 15))
p15.initial

(3, 14, 5, 6, 9, 13, 0, 2, 8, 1, 12, 4, 11, 7, 10, 15)

In [101]:
p15.actions(p15.initial)
#Podemos hacer todo porque el hueco está en la posicion 6

['Mover hueco arriba',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco derecha']

In [102]:
p15.result(p15.initial,"Mover hueco izquierda")
#Acción permitida

(3, 14, 5, 6, 9, 0, 13, 2, 8, 1, 12, 4, 11, 7, 10, 15)

In [103]:
"Mover hueco derecha" in p15.actions(p15.result(p15.initial,"Mover hueco derecha"))
#Tras haber movido una vez a la derecha estoy en la posicion 7 y no puedo volver a hacerlo

False

In [104]:
p15.check_solvability(p15.initial)

False

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

En primer lugar vamos a ver cómo funciona la búsqueda ciega (en anchura y en profundidad) para encontrar soluciones para los tres problemas anteriores: el problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [28]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

Para que la importación de search funcione el archivo search.py y utils.py tienen que estar en la misma carpeta que este notebook o poner la ruta correspondiente.

In [29]:
## comprueba la resolución del problema de las jarras con el método de búsqueda en anchura.  

In [30]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### EJERCICIO 3. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8 y de 16.  Comenta al final cómo se comportan los algoritmos en cuanto a eficiencia y resultados. 

In [31]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

#Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura 

breadth_first_tree_search(misioneros).solution()

# Realiza pruebas con los diferentes algoritmos observando el comportamiento.

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [32]:
breadth_first_graph_search(misioneros).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [33]:
depth_first_graph_search(misioneros).solution()

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1m', (1, 0)),
 ('1m1c', (1, 1))]

In [34]:
depth_first_tree_search(misioneros).solution()

KeyboardInterrupt: 

In [35]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [36]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [ ]:
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
## La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
## Esta llamada no funciona, se queda en [*]
## Si es método de busqueda en anchura es completo.. ¿no debería terminar? ¿qué crees que está pasando?
## Al ser búsqueda en anchura un método completo, esto es, que si hay solución bfs la encuentra (aunque haya ciclos y tardemos mucho), podemos deducir que el programa se queda
## en [*] porque la configuración inicial NO tiene solución. Si hubiera sido una búsqueda en profundidad otra posible razón de
## que el programa no termine o se quede en [*] sería que el dfs sin control de repetidos encontrara un ciclo 

In [37]:
# Comprueba si el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) tiene solución.
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
if p8.check_solvability(p8.initial):
    sol= breadth_first_tree_search(p8).solution()
else: 
    sol="Este estado inicial no tiene solucion"

In [38]:
sol

'Este estado inicial no tiene solucion'

In [39]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [40]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [41]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [42]:
depth_first_graph_search(estado).solution()

KeyboardInterrupt: 

In [43]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [ ]:
p15 = Quince_Puzzle((3, 0, 5 ,6, 9, 14, 13, 2, 8, 1, 12, 4, 11, 7, 10, 15))
if p15.check_solvability(p15.initial):
    sol=breadth_first_graph_search(p15).solution()
else:
    sol="No hay solución"
sol

In [44]:
### Comentarios del ejercicio 3. 
#### Se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones respecto a la resolución de los problemas con búsqueda ciega:
- Misioneros: Ambas versiones de bfs y la de dfs con control de repetidos encuentran soluciones rápido. Cabe notar que las
- soluciones que han encotrado bfs y dfs no son la misma pero con el mismo número de acciones. Sin embargo, dfs sin control de
- repetidos se queda colgado en un ciclo y no termina.
- Puzle de 8: En cuanto al bfs, no hemos notado diferencia entre la versión con control de repetidos o sin ella ya que ambas alcanzan la
- solución muy rápido. Sin embargo, con el dfs, aún siendo con control de repetidos, hemos necesitado un tiempo considerablemente
- alto. Esto se debe a que la búsqueda es a ciegas y que el orden preestablecido de las acciones no tiene porqué ser el óptimo
- para alcanzar la solución desde el estado inicial dado. Existirá una configuración inicial y un orden de las acciones de modo
- que la bfs tardará más que la dfs si la solución se encuentra, por ejemplo, en la primera rama que explora dfs.
- Puzle de 16: Para este se observa el mismo fenómeno pero con tiempos y soluciones mucho más largas. Aún usando el bfs con
- control de repetidos hemos tardado mucho en obtener la solución.

**********************************************************************************
Puedes añadir todo el espacio que necesites.

SyntaxError: illegal target for annotation (4051094020.py, line 3)

### EJERCICIO 4:  Definición de heurísticas

In [45]:
#### 4.1. Para el problema de los misioneros define una heurística y comenta sus propiedades
# Puedes utilizar alguna de las heurísticas vistas en clase. 

# Heurísticas para el problema de los misioneros.
# Heurítica 1: Relajación del problema en la cual existen infinitas barcas a cada lado. Solución sencilla hacer
# 1M1C en cada acción: h1(n) = (C+M)/2
# Heurísitca 2: Relajación del problema en la cual los caníbales no se comen a los misioneros. En cada viaje de ida y vuelta
# transportamos a una persona para que la otra vuelva con la barca. h2(n) = 2*(Ci+Mi) - orilla(n), donde orilla(n) = 1 si
# si en el estado n la barca está en la orilla i y 0 en otro caso.
# h2 NO ES ADMISIBLE !!! Estado -> (0,2,1) -> h2(n) = 2*2 - 1 = 3 > 1 = coste_real(n)
# h2'(n) = 2(C+M) - 3*orilla(n) sí es admisible.
# h2' es más informada que h1.

In [46]:
#### 4.2. Para el problema del puzle de 8 y de 16 se pide definir  las siguientes funciones heurísticas:
# linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
# manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
# max_heuristic(node: maximo de las dos anteriores
# sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 

def linear(node):
    goal = (1,2,3,4,5,6,7,8,0)
    difs = 0
    for i in range(len(goal)-1):
        if node.state[i] != i+1:
            difs += 1
    if node.state[len(goal)-1] != 0:
        difs += 1
    return difs            


def manhattan(node):
    mhd = 0
    state = node.state
    for i in range(len(state)):
        filaEsta = i // 3
        colEsta = i % 3
        if state[i] % 3 == 0:
            colIr = 2
        else:
            colIr = (state[i] % 3) - 1
        if (state[i] == 0):
            filaIr = 2
        else:
            filaIr = (state[i] - 1)//3
        
        mhd += abs(filaIr - filaEsta) + abs(colIr - colEsta)
    return mhd
                
def sqrt_manhattan(node):
    mhd = manhattan(node)
    
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        

In [47]:
 linear(Node((1, 2, 3, 4, 5, 8, 7, 6, 0)))

2

In [48]:
manhattan(Node((0,2,3,4,5,6,7,8,1)))

8

### EJERCICIO 5. 
#### Usar las implementaciones de AIMA de los algoritmos de búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema de los misioneros y algun ejemplo del puzle de 8 y puzle de 16. 

#### Comparar los costes temporales usando %timeit y comentar los resultados.

In [14]:
####  Vamos a probar el 8-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+
from search import *
import timeit

puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()


['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [50]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [51]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [52]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [53]:
astar_search(puzle,linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [54]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? ¿y en los resultados?
Vamos a medirlo 

In [55]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [56]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

5.55 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [57]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

3.1 ms ± 577 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [58]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

42.5 ms ± 2.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

3.57 ms ± 674 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [60]:
puzzle_1 = Quince_Puzzle((1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0))
puzzle_2 = Quince_Puzzle((1, 2, 3 ,4, 5, 6, 7, 8, 10, 0,11,12, 9, 13, 14, 15))
puzzle_3 = Quince_Puzzle((1, 2, 0, 3, 5, 6, 8, 4, 9, 10,7,12,13,14,11,15))

In [61]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

4.8 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

1.78 ms ± 289 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

5.58 ms ± 599 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [64]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

1.04 ms ± 90.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


####  Realiza pruebas también con el puzle de 16 con los estados iniciales que quieras.  Escribe aquí tus conclusiones sobre qué heurística te parece mejor para este problema.  

Ten en cuenta que el número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. En el caso del siguiente estado inicial que sí tiene solución: (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


#### Justifica claramente la respuesta basándote en los resultados esperados  (según las clases teóricas) y los resultados obtenidos en las pruebas anteriores.

Con el fin de Con evitar esperas extremadamente largas, hemos seleccionado ejemplos muy sencillos para el puzzle del 15. Hemos comprobado que, al igual que con el puzzle del 8, las mejores heurísticas son la de la distancia manhattan y la del máximo, siendo esta vez la de la distancia manhattan ligeramente mejor. Estos son los resultados teóricos esperados, ya que sabemos que la distancia manhattan es una heurística más informada que la lineal. Puede ser que se de algún caso en el cual la búsqueda a ciegas termine antes que estas, pero eso dependerá del estado inicial y de la "suerte" que tenga la búsqueda ciega.


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [9]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas. 

# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

In [66]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [67]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [68]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [69]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [70]:
astar_search(p8, manhattan).solution()

['Mover hueco derecha']

In [71]:
astar_search(p8p, manhattan).solution()

['Mover hueco derecha']

In [6]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """
    p8=Ocho_Puzzle(estado_inicial)
    p8p=Problema_con_Analizados(p8)
    if p8.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [73]:
%%time
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2
CPU times: user 1.11 ms, sys: 1.25 ms, total: 2.36 ms
Wall time: 6.25 ms


In [12]:
e1 = Problema_con_Analizados(Ocho_Puzzle((7,2,4,3,6,1,8,0,5)))
e2 = Problema_con_Analizados(Ocho_Puzzle((4,5,6,1,0,3,7,8,2)))
e3 = Problema_con_Analizados(Ocho_Puzzle((5,3,1,8,0,4,6,2,7)))
e4 = Problema_con_Analizados(Ocho_Puzzle((0,8,7,6,5,4,3,2,1)))
e5 = Problema_con_Analizados(Ocho_Puzzle((8,0,3,1,2,6,4,7,5)))

In [15]:
%%time
resuelve_ocho_puzzle(e1.initial,breadth_first_graph_search)

Este problema no tiene solucion. 
CPU times: user 435 µs, sys: 119 µs, total: 554 µs
Wall time: 493 µs


In [16]:
%%time
resuelve_ocho_puzzle(e2.initial,breadth_first_graph_search)

KeyboardInterrupt: 

In [17]:
%%time
resuelve_ocho_puzzle(e3.initial,breadth_first_graph_search)

KeyboardInterrupt: 

In [ ]:
%%time
resuelve_ocho_puzzle(e4.initial,breadth_first_graph_search)

In [ ]:
%%time
resuelve_ocho_puzzle(e5.initial,breadth_first_graph_search)

In [ ]:
%%time
resuelve_ocho_puzzle(e1.initial,depth_first_graph_search)

In [ ]:
%%time
resuelve_ocho_puzzle(e2.initial,depth_first_graph_search)

In [ ]:
%%time
resuelve_ocho_puzzle(e3.initial,depth_first_graph_search)

In [ ]:
%%time
resuelve_ocho_puzzle(e4.initial,depth_first_graph_search)

In [ ]:
%%time
resuelve_ocho_puzzle(e5.initial,depth_first_graph_search)

In [19]:
%%time
resuelve_ocho_puzzle(e1.initial,uniform_cost_search)

Este problema no tiene solucion. 
CPU times: user 384 µs, sys: 311 µs, total: 695 µs
Wall time: 418 µs


In [18]:
%%time
resuelve_ocho_puzzle(e2.initial,uniform_cost_search)

KeyboardInterrupt: 

In [126]:
%%time
resuelve_ocho_puzzle(e3.initial,uniform_cost_search)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 26. Nodos analizados: 164781
CPU times: user 1h 37min 15s, sys: 23.3 s, total: 1h 37min 39s
Wall time: 1h 37min 57s


In [127]:
%%time
resuelve_ocho_puzzle(e4.initial,uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 28. Nodos analizados: 177480
CPU times: user 1h 40min 14s, sys: 18 s, total: 1h 40min 32s
Wall time: 1h 40min 43s


In [128]:
%%time
resuelve_ocho_puzzle(e5.initial,uniform_cost_search)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 17. Nodos analizados: 14094
CPU times: user 1min 8s, sys: 219 ms, total: 1min 8s
Wall time: 1min 8s


In [101]:
%%time
resuelve_ocho_puzzle(e1.initial,best_first_graph_search,linear)

Este problema no tiene solucion. 
CPU times: user 276 µs, sys: 220 µs, total: 496 µs
Wall time: 584 µs


In [102]:
%%time
resuelve_ocho_puzzle(e2.initial,best_first_graph_search,linear)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hue

In [103]:
%%time
resuelve_ocho_puzzle(e3.initial,best_first_graph_search,linear)

Solución: ['Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', '

In [104]:
%%time
resuelve_ocho_puzzle(e4.initial,best_first_graph_search,linear)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 28. Nodos analizados: 29
CPU times: user 1.73 ms, sys: 115 µs, total: 1.85 ms
Wall time: 2.5 ms


In [84]:
%%time
resuelve_ocho_puzzle(e5.initial,best_first_graph_search,linear)

Solución: ['Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 35. Nodos analizados: 139
CPU times: user 12.6 ms, sys: 3.07 ms, total: 15.6 ms
Wall time: 32.6 ms


In [105]:
%%time
resuelve_ocho_puzzle(e1.initial,best_first_graph_search,manhattan)

Este problema no tiene solucion. 
CPU times: user 778 µs, sys: 403 µs, total: 1.18 ms
Wall time: 3.96 ms


In [106]:
%%time
resuelve_ocho_puzzle(e2.initial,best_first_graph_search,manhattan)

Solución: ['Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco 

In [107]:
%%time
resuelve_ocho_puzzle(e3.initial,best_first_graph_search,manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hue

In [108]:
%%time
resuelve_ocho_puzzle(e4.initial,best_first_graph_search,manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 28. Nodos analizados: 29
CPU times: user 1.16 ms, sys: 165 µs, total: 1.32 ms
Wall time: 1.24 ms


In [110]:
%%time
resuelve_ocho_puzzle(e5.initial,best_first_graph_search,manhattan)

Solución: ['Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 31. Nodos analizados: 81
CPU times: user 5.81 ms, sys: 425 µs, total: 6.24 ms
Wall time: 6.94 ms


In [111]:
%%time
resuelve_ocho_puzzle(e1.initial,astar_search,linear)

Este problema no tiene solucion. 
CPU times: user 956 µs, sys: 1.19 ms, total: 2.15 ms
Wall time: 3.48 ms


In [112]:
%%time
resuelve_ocho_puzzle(e2.initial,astar_search,linear)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 20. Nodos analizados: 3377
CPU times: user 3.2 s, sys: 33.4 ms, total: 3.23 s
Wall time: 3.3 s


In [124]:
%%time
resuelve_ocho_puzzle(e3.initial,astar_search,linear)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 26. Nodos analizados: 34703
CPU times: user 8min 51s, sys: 6.44 s, total: 8min 57s
Wall time: 10min 13s


In [125]:
%%time
resuelve_ocho_puzzle(e4.initial,astar_search,linear)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 28. Nodos analizados: 56011
CPU times: user 21min 17s, sys: 12.3 s, total: 21min 30s
Wall time: 22min 48s


In [118]:
%%time
resuelve_ocho_puzzle(e5.initial,astar_search,linear)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 17. Nodos analizados: 798
CPU times: user 190 ms, sys: 5.22 ms, total: 195 ms
Wall time: 200 ms


In [119]:
%%time
resuelve_ocho_puzzle(e1.initial,astar_search,manhattan)

Este problema no tiene solucion. 
CPU times: user 427 µs, sys: 499 µs, total: 926 µs
Wall time: 990 µs


In [120]:
%%time
resuelve_ocho_puzzle(e2.initial,astar_search,manhattan)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 20. Nodos analizados: 1153
CPU times: user 401 ms, sys: 6.42 ms, total: 408 ms
Wall time: 418 ms


In [121]:
%%time
resuelve_ocho_puzzle(e3.initial,astar_search,manhattan)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 26. Nodos analizados: 5147
CPU times: user 9.42 s, sys: 110 ms, total: 9.53 s
Wall time: 9.93 s


In [122]:
%%time
resuelve_ocho_puzzle(e4.initial,astar_search,manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 28. Nodos analizados: 805
CPU times: user 221 ms, sys: 5.76 ms, total: 226 ms
Wall time: 249 ms


In [123]:
%%time
resuelve_ocho_puzzle(e5.initial,astar_search,manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 17. Nodos analizados: 205
CPU times: user 22.2 ms, sys: 2.44 ms, total: 24.7 ms
Wall time: 24.7 ms


### Ejercicio 6:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

          E1              E2              E3              E4               E5
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+  
     | 7 | 2 | 4 |   | 4 | 5 | 6 |   | 5 | 3 | 1 |   | H | 8 | 7 |    | 8 | H | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 3 | 6 | 1 |   | 1 | 0 | 3 |   | 8 | 0 | 4 |   | 6 | 5 | 4 |    | 1 | 2 | 6 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 8 | H | 5 |   | 7 | 8 | 2 |   | 6 | 2 | 7 |   | 3 | 2 | 1 |    | 4 | 7 | 5 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo)
 
-----------------------------------------------------------------------------------------
                                       E1                  E2           E3          E4         E5
    Anchura                            L=No sol            L=20         L=26        L=28       L=17
                                       T=No sol(365µs)     T=2m45s      T=17m56s    T=18m19s   T=10.2s
                                       NA=No sol           NA=57057     NA=170050   NA=178224  NA=15755
   
    Profundidad                        L=No sol            L=39720      L=41652     L=64830    L=48525
                                       T=No sol(2.05ms)    T=6m15s      T=7m37s     T=28m42s   T=12m19s
                                       NA=No sol           NA=42926     NA=45312    NA=106784  NA=54360
                                       
    Coste Uniforme                     L=No sol            L=20         L=26        L=28       L=17
                                       T=No sol(2.35 ms)   T=19m4s      T=1h37m39s  T=1h40m32s T=1m8s
                                       NA=No sol           NA=48428     NA=164781   NA=177480  NA=14094
                                       
    Primero el mejor (linear)          L=No sol            L=76         L=102       L=28       L=35
                                       T=No sol(496 µs)    T=187ms      T=227ms     T=1.85ms   T=15.6ms
                                       NA=No sol           NA=748       NA=777      NA=29      NA=139
                                                                            
    Primero el mejor (manhattan)       L=No sol            L=74         L=144       L=28       L=31  
                                       T=No sol(1.18ms)    T=136ms      T=357ms     T=1.32ms   T=6.24ms
                                       NA=No sol           NA=591       NA=1081     NA=29      NA=81
                                                                             
    A* (linear)                        L=No sol            L=20         L=26        L=38       L=17
                                       T=No sol(2.15ms)    T=3.23ms     T=8min57s   T=21m30s   T=195ms  
                                       NA=No sol           NA=3377      NA=34703    NA=56011   NA=798
                                                                             
    A* (manhattan)                     L=No sol            L=20         L=26        L=28       L=17 
                                       T=No sol(926µs)     T=408ms      T=9.53s     T=226 ms   T=24.7ms
                                       NA=No sol           NA=1153      NA=5147     NA=805     NA=205


 -----------------------------------------------------------------------------------------
PUEDES AMPLIAR LA TABLA CON PRUEBAS ADICIONALES. 

Escribe aquí tus conclusiones 

 **justificar los resultados con las distintas propiedades teóricas estudiadas en clase**.  
 
Se puede observar que, por lo general, las búsquedas informadas son mejores en cuanto a tiempo de ejecución que las búsquedas cicegas. Esto era de esperar, ya que salvo que tengamos mucha suerte, una búsqueda informada trabajará mejor que una ciega aunque tenga que hacer cálculos de estimaciones. Entre las búsquedas informadas, se observa que la heurística manhattan funciona mejor que la lineal para este problema ya que el coste que estima es más cercano al coste real de cada instancia. Esto se puede observar en la tabla ya que, si comparamos los tiempos de linear y manhattan para un mismo algoritmo, los de manhattan son mejores en general. Por último, en este caso vemos que parece ser que las búsquedas "primero el mejor" están siendo más eficaces que las A*. Quizás esto se deba a la voracidad del algoritmo de primero el mejor, o también porque al ser A* óptimo (esto es, que la solución que devuelve es de coste óptimo) tarde más en encontrar una solución que "primero el mejor", que devuelve la primera que encuentra.


### Ejercicio 7:  resolver usando distintas búsquedas y/o heurísticas, el problema del 16 puzzle para algunos  estados iniciales:


In [124]:
def linear15(node):
    goal = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
    difs = 0
    for i in range(len(goal)-1):
        if node.state[i] != i+1:
            difs += 1
    if node.state[len(goal)-1] != 0:
        difs += 1
    return difs            


def manhattan15(node):
    mhd = 0
    state = node.state
    for i in range(len(state)):
        filaEsta = i // 4
        colEsta = i % 4
        if state[i] % 4 == 0:
            colIr = 3
        else:
            colIr = (state[i] % 4) - 1
        if (state[i] == 0):
            filaIr = 3
        else:
            filaIr = (state[i] - 1)//4
        
        mhd += abs(filaIr - filaEsta) + abs(colIr - colEsta)
    return mhd

In [125]:
puzzle_1 = Quince_Puzzle((1,3,8,6,5,0,2,4,9,10,7,11,13,14,15,12))
puzzle_2 = Quince_Puzzle((2, 6, 3 ,4, 11, 1, 10, 8, 0, 14,7,12, 5, 9, 13, 15))

In [126]:
puzzle_1.check_solvability(puzzle_1.initial)

True

In [127]:
puzzle_2.check_solvability(puzzle_2.initial)

True

In [130]:
%%time
best_first_graph_search(puzzle_1,linear15)

CPU times: user 7.71 s, sys: 127 ms, total: 7.83 s
Wall time: 8.28 s


<Node (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)>

In [131]:
%%time
best_first_graph_search(puzzle_1,manhattan15)

CPU times: user 7.09 s, sys: 87 ms, total: 7.18 s
Wall time: 8.03 s


<Node (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)>

In [132]:
%%time
astar_search(puzzle_1,linear15)

CPU times: user 5.78 ms, sys: 1.44 ms, total: 7.22 ms
Wall time: 9.39 ms


<Node (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)>

In [133]:
%%time
astar_search(puzzle_1,manhattan15)

CPU times: user 3.66 ms, sys: 203 µs, total: 3.87 ms
Wall time: 7.53 ms


<Node (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)>

In [134]:
%%time
best_first_graph_search(puzzle_2,linear15)

CPU times: user 2.48 s, sys: 26.9 ms, total: 2.51 s
Wall time: 2.58 s


<Node (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)>

In [135]:
%%time
best_first_graph_search(puzzle_2,manhattan15)

CPU times: user 22min 14s, sys: 10.7 s, total: 22min 25s
Wall time: 23min 3s


<Node (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)>

In [136]:
%%time
astar_search(puzzle_2,linear15)

CPU times: user 2.16 s, sys: 14.3 ms, total: 2.18 s
Wall time: 2.23 s


<Node (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)>

In [137]:
%%time
astar_search(puzzle_2,manhattan15)

CPU times: user 59.9 ms, sys: 2.81 ms, total: 62.7 ms
Wall time: 67 ms


<Node (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)>

                                       P1.                 P2       
    Primero el mejor (linear)          T=7.83s             T=2.51s      
                                           
    Primero el mejor (manhattan)       T=7.18s             T=22m25s     
                                                                             
    A* (linear)                        T=7.22ms            T=2.18s      
                                                                             
    A* (manhattan)                     T=3.87ms            T=62.7ms     

Habiendo visto los resultados anteriores para el 8-puzzle, decidimos probar únicamente con best_first para las dos heurísitcas y con A* para las mismas dos. Los puzzles 1 y 2 son el resultado de genenerar uno aleatorio de la aplicación de kociemba.org y dejar que se apliquen varios pasos que conducen hasta la solución, pero no todos. Esto nos ha dejado con dos estados iniciales no muy complicados que se han resuelto rápido. Los resultados obtenidos para P1 concuerdan con lo esperado ya que se ve que linear es mejor manhattan para cada algoritmo, y además A* da mejores tiempos que best_first. Los resultados para P2 nos sorprende porque, mientras que A* funciona como esperábamos, el best_first con la heurística manhattan tarda mucho en comparación con el resto de tiempos. Suponemos que es un ejemplo en el cual la estrategia devoradora dada por la heurística no funciona bien porque la solución quizás no resulte de ir cogiendo el mejor camino de heurísticas

###  PARTE IV:  Se os proporcionará un enunciado de algún problema para representar y resolver usando el paradigma del espacio de estados. 


## Alejandro Paz - Enrique Queipo de Llano
## Parte IV

In [33]:
from search import *

In [34]:
# APARTADO a)
class Muelles(Problem):
    """Problema de los muelles:
    Representaremos los estados como tuplas de 6 pilas, que a su vez serán representadas como tuplas de 1's y 0's,
    1 si el contenedor es objetivo y 0 si no lo es, las pilas 0,1,2 son las del muelle 1 y las 3,4,5 las del muelle 2
    Ademas las acciones serán representadas como tuplas (i,j) donde i es la pila de donde se coje el contenedor y j donde se 
    deja. goal_test se podría conseguir en coste logarítmico aplicando búsqueda binaria y cambiando la representación por una en la 
    que tuviera los identificadores de los contenedores junto a un valor que indique si es objetivo o no, pero consideramos 
    que no merece la pena complicar tanto la respresentación"""

    def __init__(self,initial):
        self.initial = initial
        self.numObj = 0
        self.alt = 3
        #Calcular numero de contenedores objetivo
        for i in range (len(initial)):
            for j in range(len(initial[i])):
                if (initial[i][j] == 1):
                    self.numObj +=1
        
        
    def actions(self,estado):
        #accs es una lista que inicializamos vacía, comprobaremos las precondiciones y añadiremos en esta lista las acciones aplicables al estado.
        accs=list() 
        # Para cada pila, mover de una pila a otra distinta siempre que no se supere la altura
        for i in range(len(estado)):
            for j in range(len(estado)):
                if (i != j and len(estado[i]) > 0):
                    if (len(estado[j]) + 1 <= self.alt):
                        accs.append((i,j))
        return accs
        # se devuelve en accs todas las acciones aplicables

    def result(self,estado,accion):
        # aplica una acción a un estado (esta función se llamará desde el algoritmo de búsqueda)
        desde = accion[0]
        hasta = accion[1]
        estadoNuevo = []
        for i in range(len(estado)):
            # Copiar pila antigua
            estadoNuevo.append(list(estado[i])) 
            if (i == desde): # Si muevo, hago pop
                estadoNuevo[i].pop(0)
            elif (i == hasta): # Si recibe, hago insert
                estadoNuevo[i].insert(estado[desde][0],0)
        for i in range(len(estadoNuevo)):
            estadoNuevo[i] = tuple(estadoNuevo[i])
        return tuple(estadoNuevo)


    def goal_test(self,estado):
        encontrados = 0
        # Comprobar que los contenedores objetivo estan disponibles
        for i in range(3):
            for j in range(1,len(estado[i])):
                if (estado[i-1] == 0 and estado[i] == 1):
                    return False
        #Comprobar que estan todos en el muelle 1 (no me faltan)
        for i in range(3):
            for j in range(len(estado[i])):
                encontrados += estado[i][j]
                    
        return encontrados == self.numObj
    
    
    def coste_de_aplicar_accion(self, accion):
        """Respecto a la versiòn original de AIMA hemos incluido está función que devuelve el coste de un único operador. El coste sera
        1 si se mueven contendores dentro de un mismo muelle y 3 si son en muelles distintos (coste == numero de maquinas que hay que accionar)"""
        i = accion[0]
        j = accion[1]
        if ((i <= 2 and j >= 3) or (i >= 3 and j <= 2)):
            return 3
        else:
            return 1
        
    

In [35]:
prueba = Muelles(((1,0),(0,0),(),(0,1),(0,1,0),(0,)))

In [36]:
prueba.numObj

3

In [37]:
prueba.actions(prueba.initial)

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 5),
 (1, 0),
 (1, 2),
 (1, 3),
 (1, 5),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 5),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 5),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3)]

In [38]:
prueba.result(prueba.initial,(1,0))

((0, 1, 0), (0,), (), (0, 1), (0, 1, 0), (0,))

In [39]:
prueba.goal_test(prueba.initial)

False

In [40]:
prueba.goal_test(((1,1,1),(0,),(0,),(0,),(0,),(0,)))

True

In [41]:
prueba.coste_de_aplicar_accion((2,5))

3

In [42]:
prueba.coste_de_aplicar_accion((2,1))

1

Apartado b)
Salta a la vista rápidamente la existencia de ciclos al hacer distintas acciones, ya que se podrían repetir las acciones (0,1) (1,0) constantemente (p. ej), por este motivo se descartan de primeras los algoritmos tree_search que no controlan ciclos.
Además no parece muy razonable tampoco utilizar búsquedas ciegas, por lo que implementaremos una heurística y emplearemos el algoritmo A*. También usaremos el de primero el mejor para compararlos entre sí, pero pensamos que A* cobra más sentido aquí ya que en el mundo real querremos usar las máquinas lo menos posible (menor coste)

In [43]:
# Heurística contar las cajas objetivo que tengo en el muelle 2
def cajasEn2(node):
    encontrados = 0
    for i in range(3,6):
        for j in range(len(node.state[i])):
            encontrados += node.state[i][j]

    return encontrados

# Se relaja el problema pensando que se pueden coger las cajas de cualquier posicion de la pila, no solo la cima. Así el coste
# de mover cajas objetivo del muelle 2 al 1 será 3*cajasEn2 y el de hacer que las del 1 estén disponibles será 1.
def linear(node):
    suma = 3*cajasEn2(node)
    for i in range(3):
        primerCero = False
        for j in range(len(node.state[i])):
            if (node.state[i][j] == 0):
                primerCero = True
            if (primerCero):
                suma += node.state[i][j]
    return suma  

In [44]:
cajasEn2(Node(((0,),(0,),(0,),(0,),(0,),(1,1,1))))

3

In [45]:
linear(Node(((0,0,1),(0,),(1,0,0),(0,),(0,),(0,1,0))))

4

In [53]:
%%time
astar_search(prueba,cajasEn2)

CPU times: user 1min 7s, sys: 1.96 s, total: 1min 9s
Wall time: 1min 18s


In [54]:
%%time
astar_search(prueba,linear)

CPU times: user 56.9 s, sys: 1.53 s, total: 58.4 s
Wall time: 1min 10s


In [55]:
%%time
best_first_graph_search(prueba,cajasEn2)

CPU times: user 8.22 s, sys: 218 ms, total: 8.44 s
Wall time: 9.73 s


In [56]:
%%time
best_first_graph_search(prueba,linear)

CPU times: user 8.15 s, sys: 214 ms, total: 8.36 s
Wall time: 9.41 s


Se observa rápidamente que la heurística linear parece ser mejor que la de cajasEn2, como era de esperar. Como nos ha ocurrido antes con el 8-puzzle, resulta que el algoritmo best_first_graph_search encuentra antes una solución de forma voraz utilizando la heurística. Dependerá de la importancia del coste del problema ver si preferimos encontrar una solución subóptima más rápido (por ejemplo, si hay mucha prisa en que lleguen los barcos para recoger los contenedores, quizás convenga utilizar best_first), o bien encontrar la óptima tardando un poco más (por ejemplo, si el coste en euros de la electricidad utilizada para mover lás máquinas es muy alta, quizás convenga usar A*).

Apartado d)

-- 1 -- NUEVOS CONTENEDORES: Incluir nuevos contenedores, ya sean objetivos o no, es una modificación que no afecta al factor de ramificación del problema al no generar nuevas acciones. Esto hace que los árboles que se generarán sea  igual de anchos pero más profundos, pues ahora hay mas contenedores que mover y por tanto es necesario realizar más pasos hasta llegar a la solución.  Con respecto a las soluciones, estás tendrán un coste superior (pues por lo general hay que realizar mas acciones para alcanzar el estado objetivo) pero el tiempo que tardaremos en encontrarlas no será muy superior al actual al no verse modificado el factor de ramificación

-- 2 -- MODIFICAR ALTURAS: El hecho de modificar las alturas de las pilas se traduce en una modificación del factor de ramificación efectivo del problema, pero no en el factor de ramificación general del mismo. Si hacemos que las pilas puedan tener altura 4 en lugar de 3 , aunque no aparece ninguna acción nueva (es decir, el factor de ramificación no se ve alterado), la acción de mover un contenedor de una pila a otra que ya tiene altura 3 pasa de no ser posible a sí serlo y, en consecuencia, el factor de ramificación efectivo es mayor (se parece más al factor de ramificación del problema) en este caso. Esto se traduce en una exploración más exhaustiva de los árboles, pero ahora podremos alcanzar la solución en menos pasos (pues hacer pilas más altas nos evita pasos intermedios) y por lo tanto, si conseguimos una heurística capaz de reconocer los nodos más prometedores encontraremos soluciones de muy poco coste en un tiempo razonable. Es decir, dependerá de si somos capaces de encontrar un algoritmo "voraz" efectivo que podamos encontrar una solución más rápido siguiendo la estrategia de "aprovechar la altura nueva", pero si utilizamos algoritmos de exploración del árbol, tardaremos más. Si permitimos que las pilas tengan menos altura el comportamiento es inverso, el factor de ramificación efectivo será menor pero necesitaremos más pasos para "desbloquear" los contenedores efectivos, lo cual se traduce en soluciones de más coste que se generan en un tiempo menor.

Apartado e)

-- 1 -- VARIAR NÚMERO DE PILAS: Suponemos que el número de pilas en cada muelle es el mismo. Si añadimos pilas nuevas, el factor de ramificación aumentará dado que cuento con más acciones posibles, lo cual significa que tendremos árboles más anchos y que su exploración, a priori, será más costosa en tiempo. Sin embargo, el hecho de tener más pilas en el muelle 1 nos ayuda considerablemente, ya que nos permitirá colocar también en ellas contenedores objetivo y ahorrarnos acciones que antes eran inevitables. Pensamos que esto se traducirá en que, a pesar de que el árbol sea más ancho, encontraremos soluciones a menor profundidad, por lo que elaborando una heurística "más inteligente" que saque provecho de dicha holgura que nos proporciona tener más pilas, quizás tardemos menos que haciendo una gran exploración. La dificultad reside en encontrar dicha heurística, ya que usando las que hemos implementado, pensamos que aumentando el número de pilas, aumentarían los tiempos.
Por otro lado, si disminuimos el número de pilas, el factor de ramificación se hará más pequeño, pero a la inversa del caso anterior, tendremos que maniobrar mucho con los contenedores (a lo torres de hanoi) para colocar los objetivo en el muelle 1, con lo que tendremos un árbol más profundo y menos ancho.

-- 2 -- AÑADIR NUEVOS MUELLES: Si el muelle que añadimos no es un muelle objetivo, es decir, no hay que colocar en él los contenedores objetivo, podemos verlo como si al muelle 2 le hubiéramos dado más pilas, con lo que estaríamos en el caso anterior. Con esto, el factor de ramificación aumenta, pero vuelve a ocurrir lo de antes: tengo más sitios donde depositar los contendores que no quiero, luego quizás siendo inteligentes tengamos una solución a mano más rápida.
Si el muelle que añadimos es objetivo, se puede ver de nuevo como que hemos añadido al muelle 1 más pilas, teniendo más lugares donde depositar los contendores objetivo aunque también aumentamos el factor de ramificación.
Cabe mencionar que este argumento nos sirve para pensar en la resolución del problema, pero no en el coste de las soluciones. Tener un nuevo muelle aumentará este coste porque tendremos que utilizar más cintas transportadoras y quizás no todos los muelles estén conectados entre sí. Por ejemplo, si no hay cinta entre 1 y 3, el coste de llevar un 
contenderor desde 3 hasta 1 será el coste de llevarlo de 3 a 2 y luego de 2 a 1, es decir un total de 6 en vez de 3.